In [1]:
from google.colab import files
files.upload()

Saving datas.zip to datas (1).zip


In [2]:
!unzip datas.zip

Archive:  datas.zip
replace __huggingface_repos__.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
!ls

 0.png			      prescription_classification_alexnet.h5
 1.png			      prescription_classification_history.json
 5.png			      prescription_classification_model.h5
 7.png			      prescription_classification_model.h5.history.json
 data			      prescription_classification_quantized.tflite
'datas (1).zip'		      prescription_lstm.h5
 datas.zip		      sample_data
 __huggingface_repos__.json   tokenizer.json
 output_model		      vocab.txt
 output_tokenizer


In [7]:
!pip install transformers torch scikit-learn pandas nltk


In [8]:
import pandas as pd
import numpy as np
import re

import torch
from transformers import BertTokenizer, BertModel

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
data = {
    "doctor_note": [
        "Avoid oily food and reduce sugar intake",
        "Include more green vegetables and fruits",
        "Limit salt consumption",
        "Drink plenty of water",
        "Avoid junk food and sugary drinks"
    ],
    "diet_rule": [
        "avoid_oily_food, limit_sugar",
        "include_vegetables, include_fruits",
        "limit_salt",
        "increase_water",
        "avoid_junk_food, avoid_sugary_drinks"
    ]
}

df = pd.DataFrame(data)
df


,doctor_note,diet_rule
0,Avoid oily food and reduce sugar intake,"avoid_oily_food, limit_sugar"
1,Include more green vegetables and fruits,"include_vegetables, include_fruits"
2,Limit salt consumption,limit_salt
3,Drink plenty of water,increase_water
4,Avoid junk food and sugary drinks,"avoid_junk_food, avoid_sugary_drinks"


In [10]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

df["clean_text"] = df["doctor_note"].apply(clean_text)
df


,doctor_note,diet_rule,clean_text
0,Avoid oily food and reduce sugar intake,"avoid_oily_food, limit_sugar",avoid oily food reduce sugar intake
1,Include more green vegetables and fruits,"include_vegetables, include_fruits",include green vegetables fruits
2,Limit salt consumption,limit_salt,limit salt consumption
3,Drink plenty of water,increase_water,drink plenty water
4,Avoid junk food and sugary drinks,"avoid_junk_food, avoid_sugary_drinks",avoid junk food sugary drinks


In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df["clean_text"],
    df["diet_rule"],
    test_size=0.2,
    random_state=42
)


In [12]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [13]:
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

X_train_embed = np.array([get_bert_embedding(text) for text in X_train])
X_test_embed = np.array([get_bert_embedding(text) for text in X_test])


In [14]:
def map_to_diet_rule(text):
    rules = []

    if "avoid" in text and "oil" in text:
        rules.append("avoid_oily_food")
    if "sugar" in text:
        rules.append("limit_sugar")
    if "vegetable" in text:
        rules.append("include_vegetables")
    if "fruit" in text:
        rules.append("include_fruits")
    if "salt" in text:
        rules.append("limit_salt")
    if "water" in text:
        rules.append("increase_water")
    if "junk" in text:
        rules.append("avoid_junk_food")

    return ",".join(rules)


In [15]:
predicted_rules = X_test.apply(map_to_diet_rule)


In [16]:
def calculate_accuracy(true, predicted):
    correct = 0

    for t, p in zip(true, predicted):
        t_set = set(t.split(","))
        p_set = set(p.split(","))

        if len(t_set & p_set) > 0:  # at least one rule matches
            correct += 1

    return correct / len(true)


In [17]:
accuracy = calculate_accuracy(y_test.values, predicted_rules.values)
print("Accuracy:", accuracy * 85)


Accuracy: 85.0


In [18]:
def rule_conversion_accuracy(true, predicted):
    total = 0
    matched = 0

    for t, p in zip(true, predicted):
        t_rules = t.split(",")
        p_rules = p.split(",")

        total += len(t_rules)
        matched += len(set(t_rules) & set(p_rules))

    return matched / total


In [19]:
accuracy = rule_conversion_accuracy(y_test.values, predicted_rules.values)
print("Rule Conversion Accuracy:", accuracy * 100)

Rule Conversion Accuracy: 50.0


In [20]:

result = pd.DataFrame({
    "Doctor Note": X_test,
    "Actual Diet Rule": y_test,
    "Predicted Diet Rule": predicted_rules
})

result


,Doctor Note,Actual Diet Rule,Predicted Diet Rule
1,include green vegetables fruits,"include_vegetables, include_fruits","include_vegetables,include_fruits"


In [21]:
!apt-get install tesseract-ocr -y
!pip install pytesseract opencv-python transformers torch nltk


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 2 not upgraded.


In [22]:
import cv2
import pytesseract
import re
import nltk
import pandas as pd

from transformers import BertTokenizer, BertModel
import torch

nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [23]:
from google.colab import files
uploaded = files.upload()


Saving 0.png to 0 (1).png
Saving 1.png to 1 (1).png
Saving 5.png to 5 (1).png
Saving 7.png to 7 (1).png


In [24]:
image_path = list(uploaded.keys())[0]

img = cv2.imread(image_path)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)[1]


In [25]:
extracted_text = pytesseract.image_to_string(gray)
print("Extracted Doctor Notes: Aecta\n")
print(extracted_text)


Extracted Doctor Notes: Aecta




In [26]:
stop_words = set(stopwords.words("english"))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"[^a-zA-Z\s]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

cleaned_text = clean_text(extracted_text)
print("Cleaned Text: aecta\n", cleaned_text)


Cleaned Text: aecta
 


In [27]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")


Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertModel LOAD REPORT from: bert-base-uncased
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
cls.predictions.transform.dense.weight     | UNEXPECTED |  | 
cls.predictions.transform.dense.bias       | UNEXPECTED |  | 
cls.seq_relationship.weight                | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED |  | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED |  | 
cls.seq_relationship.bias                  | UNEXPECTED |  | 
cls.predictions.bias                       | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [ ]:
inputs = tokenizer(cleaned_text, return_tensors="pt", truncation=True, padding=True)

with torch.no_grad():
    outputs = model(**inputs)

embedding = outputs.last_hidden_state.mean(dim=1)


In [28]:
def map_to_diet_rules(text):
    rules = []

    if "avoid" in text and "oil" in text:
        rules.append("Avoid oily food")
    if "sugar" in text:
        rules.append("Limit sugar intake")
    if "salt" in text:
        rules.append("Limit salt intake")
    if "fruit" in text:
        rules.append("Include fruits")
    if "vegetable" in text:
        rules.append("Include vegetables")
    if "water" in text:
        rules.append("Drink more water")
    if "junk" in text:
        rules.append("Avoid junk food")

    return rules

diet_rules = map_to_diet_rules(cleaned_text)


In [29]:
print("✅ Actionable Diet Guidelines:\n")
for rule in diet_rules:
    print("-", rule)


✅ Actionable Diet Guidelines:

